<a href="https://colab.research.google.com/github/zrghassabi/Convolutional-Neural-Network/blob/main/dataloader_CNN3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataSet

In [ ]:
import torch
import torchvision
from torchvision import transforms


#load Dataset
train_dataset=torchvision.datasets.MNIST(root='./Dataset/',train=True, transform=transforms.ToTensor(),download=True)
valid_dataset=torchvision.datasets.MNIST(root='./Dataset/',train=False, transform=transforms.ToTensor(),download=True)
test_dataset=torchvision.datasets.MNIST(root='./Dataset/',train=False, transform=transforms.ToTensor(),download=True)

# #or apply different transforms to input data
# train_dataset=torchvision.datasets.MNIST(root='./Dataset/',train=True, transform=transforms.Compose([ 
#                                                                            transforms.Resize((14,14),transforms.InterpolationMode.BICUBIC),
#                                                                            transforms.Grayscale(),
#                                                                            transforms.ToTensor()]),
#                                                                        download=True)
# valid_dataset=torchvision.datasets.MNIST(root='./Dataset/',train=False, transform=transforms.Compose([ 
#                                                                            transforms.Resize((14,14),transforms.InterpolationMode.BICUBIC),
#                                                                            transforms.Grayscale(),
#                                                                            transforms.ToTensor()]),
#                                                                        download=True)
# test_dataset=torchvision.datasets.MNIST(root='./Dataset/',train=False, transform=transforms.Compose([ 
#                                                                            transforms.Resize((14,14),transforms.InterpolationMode.BICUBIC),
#                                                                            transforms.Grayscale(),
#                                                                            transforms.ToTensor()]),
#                                                                        download=True)



In [ ]:
train_dataset.train_data  #to see train_data

In [ ]:
train_dataset.train_data.shape  #shape of all data

In [ ]:
train_dataset.train_data[0,:,:].shape  #shape of one of data

In [ ]:
# to show one image (requires opencv)
import cv2
import numpy as np

cv2.imshow('image',np.asarray(train_dataset.train_data[0,:,:]))
cv2.waitkey(2000)

In [ ]:
train_dataset.train_labels  #to see labels

In [ ]:
#parameters
batch_size=64


In [ ]:
#data loader

train_loader=torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader=torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#main loop
for data,label in train_loader:  #to show batches with 64 size
    print(label)

In [ ]:
#main loop
for imgs,label in train_loader:  #to show batches with 64 size
    print(imgs.shape)

In [ ]:
data.shape  #pay attention that dimension is 4

In [ ]:
#load custom dataset, use Imagefolder or DatasetsFolder
#this gives you address of images in the root folder
#when you use dataloader, you will get access to images (e.x. 64 images with random idx)
#train_dataset=torchvision.datasets.DatasetFolder('',transforms=transforms.ToTensor())
#validation_dataset=torchvision.datasets.DatasetFolder('',transforms=transforms.ToTensor())
#test_dataset=torchvision.datasets.DatasetFolder('',transforms=transforms.ToTensor())

#train_dataset.samples
#train_datset.classes
#train_dataset.class_to_idx

# Model

In [ ]:
# #convolutional neural network
# # do not use this way

# import torch.nn as nn
# n_class=10
# class convnet(nn.Module):
#   def __init__(self):
#       self.conv1=nn.Conv2d(1,16,3,1,2)  #in channel, out channel, filtersize, stride, padding
#       self.BatchN1=nn.BatchNorm2d(16)
#       self.relu1=nn.Relu()
#       self.maxpl=nn.MaxPool2d(2,2)
#       self.conv2=nn.Conv2d(16,32,3,1,2)  
#       self.BatchN2=nn.BatchNorm2d(32)
#       self.relu2=nn.Relu()
#       self.maxp2=nn.MaxPool2d(2,2)
#       self.fc=nn.Linear(7*7*32, n_class) 
#   def forward(self,x):
#        a=self.conv1(x)
#        a2=self.BatchN1N1(a)
#        a3=self.relu1(a2)
#        y=self.maxp1(a3)
      
#        return y





In [ ]:
import torch.nn as nn
n_class=10



class convnet(nn.Module):
  def __init__(self):
    super(convnet,self).__init__() 
    self.layer1=nn.Sequential(nn.Conv2d(1,16,3,1,2),   
                                nn.BatchNorm2d(16),
                                nn.ReLU(),
                                nn.MaxPool2d(2,2))
     
    self.layer2=nn.Sequential(nn.Conv2d(16,32,3,1,2),
                                nn.BatchNorm2d(32),
                                nn.ReLU(),
                                nn.MaxPool2d(2,2))
     
    
    #self.fc=nn.Linear(8*8*32, 10) #10 is num_class  28by 28 is 8 by 8
    #2 times pooling that split image size (1/2), first image is 15 by 15 (cuz we have padding)
    #second image is 8 by 8. without padding, instead of 8 by 8 for FC, we have to put 7 by 7
    self.fc=nn.Linear(8*8*self.layer2[0].out_channels, 10) #10 is num_class  28by 28 is 8 by 8
  def forward(self,x):
       out1=self.layer1(x)   #this like a dict
       out2=self.layer2(out1)
       out2=out2.reshape(out2.size(0),-1) #keep batch size of first dimention
       y= self.fc(out2)
       return y

In [ ]:
convmodel=convnet() #build model
print('*convmodel:\n',convmodel)
print('*convmodel.layer1:\n',convmodel.layer1)
print('*convmodel.layer1[0]:\n',convmodel.layer1[0])
#self.layer1[2] RELU()

#out2.size(0)
#out2.reshape(out2.size(0),-1).shape


In [ ]:
print(convmodel.layer2[3].kernel_size)
print(convmodel.layer2[0].in_channels)
print(convmodel.layer2[0].out_channels)
print(convmodel.layer2[0])

In [ ]:
for imgs, lbls in train_loader:
    out=convmodel(imgs)

In [ ]:
print(out[31])
print(lbls[31])
print(imgs[31].shape)

In [ ]:
print(out[31][0])
print(out[31][3])

# train, validation,test

In [ ]:
# Params
num_epochs = 100


#loss
loss_fn=nn.CrossEntropyLoss()

# Optimizer
lr=0.001
optimizer_fn = torch.optim.Adam(convmodel.parameters(), lr=lr)
#LR
lr_sch=torch.optim.lr_scheduler.StepLR(optimizer_fn, 5,gamma=0.5) #adjust learning rate

valid_num_steps=len(valid_loader)
num_steps=len(train_loader)

for i in range(num_epochs):
    convmodel.train()
    # here you can adjust learning rate
    # if condition on d_loss < 0.2: #do this when STD of loss does not change
    #    lr_sch.step() #here count, after 5 times, it changes LR
    #print(lr_sch.get_lr())
    for j, (imgs, lbls) in enumerate(train_loader):
        out = convmodel(imgs)
        loss_val = loss_fn(out, lbls)
        optimizer_fn.zero_grad()
        loss_val.backward()
        optimizer_fn.step()
        # if (j+1) % 2 == 0:
        #     print('Train, Epoch [{}/{}] Step [{}/{}] Loss: {:.4f}'.
        #           format(i+1, num_epochs, j+1, num_steps, loss_val.item()))
        # if j == 10:
        #     break
    convmodel.eval()
    corrects = 0
    for k, (imgs, lbls) in enumerate(valid_loader):
        out = convmodel(imgs)
        loss_val = loss_fn(out, lbls)
        predicted = torch.argmax(out, 1)
        corrects += torch.sum(predicted == lbls)
        # print('Validation, Step [{}/{}] Loss: {:.4f} Acc: {:.4f} '.format(k + 1, valid_num_steps, loss_val.item(), 100. * corrects / ((k + 1) * batch_size)))

convmodel.eval()
corrects = 0
num_steps = len(test_loader)
for j, (imgs, lbls) in enumerate(test_loader):
    out = convmodel(imgs)
    predicted = torch.argmax(out, 1)
    corrects += torch.sum(predicted == lbls)
    print('Step [{}/{}] Acc {:.4f}: '.format(j+1, num_steps, 100.*corrects/((j+1)*batch_size)))

torch.save()